# Webscraping 👩🏼‍💻

En este jupyter notebook se encuentra todo el proceso de exportación de datos y su posterior limpieza utilizando la herramienta de webscraping. Las fuentes empleadas han sido Wikipedia y Indexmundi

*Consideraciones*

De nuevo, sólo se tendrán en cuenta los 11 países seleccionados para el estudio: España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva Zelanda, Reino Unido.

## Índice 📎

1. Importación de librerías y funciones
2. Extracción de datos
3. Limpieza de datos
4. Exportación del dataset final
5. Extra

## 1. Importación de librerías y funciones 📚

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 2. Extracción de datos 🎣

Con el propósito de enriquecer nuestra base de datos se procederá a extraer los datos en materia al índice de obesidad de los países seleccionados para el caso de estudio. Para ello la fuente escogida será Wikipedia: List of countries by obesity rate. 

In [2]:
url_obesidad = "https://en.wikipedia.org/wiki/List_of_countries_by_obesity_rate"

In [3]:
html_obesidad = requests.get(url_obesidad)

In [4]:
soup = BeautifulSoup(html_obesidad.content, "html.parser")

Primero debemos seleccionar los datos que queremos obtener (en nuestro caso serían los referentes a la primera tabla). Empezamos inspeccionando la página, y localizando la tabla que queremos obtener en el código de html.

*(Véase __[Tabla1]("https://user-images.githubusercontent.com/64830147/122656508-8d6ea580-d15b-11eb-97b1-ae8f294900bc.png")__)*

Observamos que se encuentra dentro de un tag llamado "table". Abrimos la consola y mediante la siguiente función la filtramos para comprobar si mediante ese filtro podemos seleccionar la información deseada: 

document.querySelectorAll("Table").forEach(elm => elm.style.background = "red")

Filtramos y selecionamos por el tag ➡️ "table". Nos encontramos con el problema de que existen dos tablas de las cuales yo sólo necesito la primera. 


Volviendo al código html observamos que seguido del tag hay una clase, por la cual podemos seleccionar. Introducimos de nuevo la función en la consola pero esta vez añadiendo el nombre de la clase, obteniendo así la tabla deseada.

document.querySelectorAll('table.wikitable.sortable').forEach(elm => elm.style.background = "blue")

*Seleccionamos la tabla empleamos la función soup ➡️ soup.find_all(name=tag_name, class_=class_name)*

In [5]:
table = soup.find_all("table",{"class":"wikitable sortable"})[0]
#Con css selectors
#table = soup.select("wiki sortable") 

A continuación seleccionamos las filas a través del tag "tr": 

In [6]:
rows = table.find_all("tr")

Reorganizamos la tabla y ajustamos el diseño de los datos para más tarde poder construir el dataset:

In [7]:
#rows = [row.text.strip().split("\n") for row in rows]
rows = [row.text.replace("\n\n", ",").replace("\n", ",").replace("\xa0", "").strip(",").split(",") for row in rows]
rows [:2]

[['Country', 'Rank', 'Obesity rate% (2016)'], ['Nauru', '1', '61.00']]

Creamos el dataset:

In [8]:
nombre_columnas = rows[0]
data = rows[1:]
obesidad = pd.DataFrame(data, columns=nombre_columnas)
obesidad.head(10)

,Country,Rank,Obesity rate% (2016)
0,Nauru,1,61.00
1,Cook Islands,2,55.90
2,Palau,3,55.30
3,Marshall Islands,4,52.90
4,Tuvalu,5,51.60
5,Niue,6,50.00
6,Tonga,7,48.20
7,Samoa,8,47.30
8,Kiribati,9,46.00
9,Federated States of Micronesia,10,45.80


In [9]:
#Exportamos la base de datos original para evitar perder datos
obesidad_data0 = obesidad
obesidad_data0.to_csv("./obesidad_data0.csv")

## 3. Limpieza de datos 🧹

In [10]:
#Importamos el dataset
obesidad2 = pd.read_csv("./obesidad_data0.csv")
obesidad2.columns =  obesidad2.columns.str.rstrip() #Elimina los espacios al final de cada título en las columnas

In [11]:
obesidad2

,Unnamed: 0,Country,Rank,Obesity rate% (2016)
0,0,Nauru,1,61.0
1,1,Cook Islands,2,55.9
2,2,Palau,3,55.3
3,3,Marshall Islands,4,52.9
4,4,Tuvalu,5,51.6
...,...,...,...,...
186,186,Cambodia,187,3.9
187,187,India,188,3.9
188,188,Timor-Leste,189,3.8
189,189,Bangladesh,190,3.6


Para empezar eliminaremos la columna de "Rank" puesto que dicha información no resulta necesaria para nuestro estudio. 

In [12]:
obesidad2 = obesidad.drop(["Rank"], axis=1)
obesidad2

,Country,Obesity rate% (2016)
0,Nauru,61.00
1,Cook Islands,55.90
2,Palau,55.30
3,Marshall Islands,52.90
4,Tuvalu,51.60
...,...,...
186,Cambodia,3.90
187,India,3.90
188,Timor-Leste,3.80
189,Bangladesh,3.60


Renombramos las columnas para facilitar su legibilidad

In [13]:
columnas = ["country","%obesity"]

In [14]:
obesidad2.columns = columnas
obesidad2

,country,%obesity
0,Nauru,61.00
1,Cook Islands,55.90
2,Palau,55.30
3,Marshall Islands,52.90
4,Tuvalu,51.60
...,...,...
186,Cambodia,3.90
187,India,3.90
188,Timor-Leste,3.80
189,Bangladesh,3.60


Procedemos a quedarnos con los 11 países seleccionados: España, Francia, Italia, Grecia, Turquía, Estados Unidos, China, India, México, Nueva Zelanda y Reino Unido. 

In [15]:
España = obesidad2[obesidad2["country"]=="Spain"]
Francia = obesidad2[obesidad2["country"]=="France"]
Italia = obesidad2[obesidad2["country"]=="Italy"]
Grecia = obesidad2[obesidad2["country"]=="Greece"]
Turquia = obesidad2[obesidad2["country"]=="Turkey"]
USA = obesidad2[obesidad2["country"]=="United States"]
China = obesidad2[obesidad2["country"]=="China"]
India = obesidad2[obesidad2["country"]=="India"]
Mexico = obesidad2[obesidad2["country"]=="Mexico"]
Nueva_Zelanda = obesidad2[obesidad2["country"]=="New Zealand"]
Reino_Unido = obesidad2[obesidad2["country"]=="United Kingdom"]

paises = [España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva_Zelanda, Reino_Unido]

In [16]:
obesidad2 = pd.concat(paises)#Filtramos los países seleccionados
obesidad2

,country,%obesity
61,Spain,23.80
86,France,21.60
106,Italy,19.90
53,Greece,24.90
16,Turkey,32.10
11,United States,36.20
168,China,6.20
187,India,3.90
28,Mexico,28.90
21,New Zealand,30.80


In [17]:
obesidad2 = obesidad2.reset_index(drop=True)#Reestructuramos el índice
obesidad2

,country,%obesity
0,Spain,23.80
1,France,21.60
2,Italy,19.90
3,Greece,24.90
4,Turkey,32.10
5,United States,36.20
6,China,6.20
7,India,3.90
8,Mexico,28.90
9,New Zealand,30.80


## Exportación del dataset final 🚀

In [18]:
obesidad_data = obesidad2
obesidad_data.to_csv("./obesidad_data.csv") #Exportación del DataFrame

## Extra ⭐

A propósito de enriquecer más nuestro estudio, procederemos a extraer los datos en materia a la diabetes para analizar si resulta también un factor de riesgo para el COVID-19. La metodología empleada ha sido la misma que para la información de la obesidad. La fuente de datos escogida ha sido Indexmundi. 

In [19]:
#Selección de los datos
url_diabetes = "https://www.indexmundi.com/facts/indicators/SH.STA.DIAB.ZS/rankings"
html_diabetes = requests.get(url_diabetes)
soup = BeautifulSoup(html_diabetes.content, "html.parser")

#document.querySelectorAll("Table").forEach(elm => elm.style.background = 'red')
table2 = soup.find("table")

#Reorganización de la tabla
rows2 = table2.find_all("tr")
rows2 = [row.text.replace("\n\n", ",").replace("\n", ",").replace("\xa0", "").strip(",").split(",") for row in rows2]
rows2 [:2]

[['Rank', 'Country', 'Value', 'Year'], ['1', 'Kiribati', '22.50', '2019']]

In [20]:
#Creación del dataset
nombre_columnas2 = rows2[0]
data2 = rows2[1:]
diabetes = pd.DataFrame(data2)
#diabetes.head(10)

In [21]:
#Renombramos las columnas
diabetes.columns = ["rank","country","%diabetes","year","none"]

In [22]:
#Eliminamos las columnas que no necesitamos
diabetes2 = diabetes.drop (["rank","year","none"],axis=1)
diabetes2

,country,%diabetes
0,Kiribati,22.50
1,Sudan,22.10
2,Tuvalu,22.10
3,Mauritius,22.00
4,New Caledonia,21.80
...,...,...
190,Senegal,2.40
191,Greenland,2.10
192,The Gambia,1.90
193,Zimbabwe,1.80


In [23]:
#Filtramos por los países seleccionados
España = diabetes2[diabetes2["country"]=="Spain"]
Francia = diabetes2[diabetes2["country"]=="France"]
Italia = diabetes2[diabetes2["country"]=="Italy"]
Grecia = diabetes2[diabetes2["country"]=="Greece"]
Turquia = diabetes2[diabetes2["country"]=="Turkey"]
USA = diabetes2[diabetes2["country"]=="United States"]
China = diabetes2[diabetes2["country"]=="China"]
India = diabetes2[diabetes2["country"]=="India"]
Mexico = diabetes2[diabetes2["country"]=="Mexico"]
Nueva_Zelanda = diabetes2[diabetes2["country"]=="New Zealand"]
Reino_Unido = diabetes2[diabetes2["country"]=="United Kingdom"]

paises2 = [España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva_Zelanda, Reino_Unido]

diabetes2 = pd.concat(paises2)
diabetes2

,country,%diabetes
94,Spain,6.90
155,France,4.80
151,Italy,5.00
156,Greece,4.70
40,Turkey,11.10
42,United States,10.80
61,China,9.20
45,India,10.40
19,Mexico,13.50
108,New Zealand,6.20


In [24]:
#Reestructuramos el índice
diabetes2 = diabetes2.reset_index(drop=True)
diabetes2

,country,%diabetes
0,Spain,6.90
1,France,4.80
2,Italy,5.00
3,Greece,4.70
4,Turkey,11.10
5,United States,10.80
6,China,9.20
7,India,10.40
8,Mexico,13.50
9,New Zealand,6.20


In [25]:
#Exportamos el dataframe
diabetes_data = diabetes2
diabetes_data.to_csv("./diabetes_data.csv")